In [1]:
!pip install -q ktrain

In [2]:
import pandas as pd 
import numpy as np 
import re
import string 
import spacy
import xgboost as xgb
from sklearn.model_selection import train_test_split
from transformers import AutoModel, AutoTokenizer, ViTFeatureExtractor, ViTModel, SwinModel, AutoImageProcessor
import tensorflow as tf
import tensorflow.keras.backend as K
import lightgbm as lgb
import warnings
from PIL import Image
from tensorflow.keras import activations

warnings.filterwarnings("ignore")

#nlp = spacy.load("en_core_web_sm")

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/dask/dataframe/_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 11.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


In [3]:
Task1_Train = pd.read_csv("/kaggle/input/multimodal-daatset-for-memes-detection/Task1Multimodal Ready.csv", encoding='ISO-8859-1')
Task1_Dev = pd.read_csv("/kaggle/input/multimodal-daatset-for-memes-detection/stA_test.csv")

In [4]:
def remove_emojis(text):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # Emoticons
                               u"\U0001F300-\U0001F5FF"  # Symbols & Pictographs
                               u"\U0001F680-\U0001F6FF"  # Transport & Map Symbols
                               u"\U0001F700-\U0001F77F"  # Alphanumeric Supplement
                               u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
                               u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
                               u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
                               u"\U0001FA00-\U0001FA6F"  # Chess Symbols
                               u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
                               u"\U00002702-\U000027B0"  # Dingbats
                               u"\U000024C2-\U0001F251" 
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

def remove_punctuation(input_string):
    translator = str.maketrans('', '', string.punctuation)
    
    result = input_string.translate(translator)
    
    return result

def remove_non_english_chars(input_string):
    translation_table = dict.fromkeys(
        i for i in range(sys.maxunicode) if unicodedata.category(chr(i)).startswith('C')
    )

    result = input_string.translate(translation_table)

    return result

def correct_spelling(input_text):
    try:
        # Create a SpellChecker object
        spell = SpellChecker()

        # Split the input text into words
        words = input_text.split()

        # Find misspelled words
        misspelled = spell.unknown(words)

        # Correct misspelled words
        corrected_text = " ".join(spell.correction(word) if word in misspelled else word for word in words)

        return corrected_text

    except Exception as e:
        print(f"An error occurred: {e}")
        return input_text
    
hyperlink_pattern = r'https?://\S+|www\.\S+'

def PreProcessTweets(tweet):
  tweet2 = str(tweet)
  tweet2 = re.sub(r'^RT[\s]+', '', tweet)
  tweet2 = re.sub(r'#\w+', '', tweet2)
  tweet2 = tweet2.replace("LINK","")
  tweet2 = tweet2.replace("&amp","") 
  tweet2 = re.sub(r'@\w+', '', tweet2)
  tweet2 = tweet2.replace("\n","")
  tweet2 = re.sub(r'_', ' ', tweet2)
  tweet2 = tweet2.replace("UN","United Nations ")
  tweet2 = tweet2.replace("imglipcom","")
  tweet2 = re.sub(hyperlink_pattern, '', tweet2)
  tweet2 = tweet2.replace(":","") 
  tweet2 = remove_emojis(tweet2)
  tweet2 = remove_punctuation(tweet2)
  tweet2 = re.sub(' +', ' ',tweet2)
  tweet2 = tweet2.replace("hes","he is")
  tweet2 = tweet2.replace("govt","Government")
  #tweet2 = correct_spelling(tweet2)
  return tweet2

Task1_Train['text'] = Task1_Train['text'].apply(lambda x: PreProcessTweets(x))
Task1_Dev['text'] = Task1_Dev['text'].apply(lambda x: PreProcessTweets(x))

In [5]:
HatePath ='/kaggle/input/subtaska-hate-speach-detection/subTaskA-20231103T134949Z-001/subTaskA/Hate Speech/'
NonHatePath = '/kaggle/input/subtaska-hate-speach-detection/subTaskA-20231103T134949Z-001/subTaskA/No Hate Speech/'
TestPath = '/kaggle/input/subtaska-hate-speach-detection/subtaskA-20231211T122329Z-001/subtaskA/'

for i in range(len(Task1_Train)):
    if(Task1_Train['label'][i] == 1):
        Task1_Train['filename'][i] = HatePath + Task1_Train['filename'][i].split('/')[-1]
    else:
        Task1_Train['filename'][i] = NonHatePath + Task1_Train['filename'][i].split('/')[-1] 
        
for i in range(len(Task1_Dev)):
    Task1_Dev['filename'][i] = TestPath + Task1_Dev['filename'][i].split('/')[-1]

In [6]:
%%time
import torch 

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
image_processor = AutoImageProcessor.from_pretrained("microsoft/swin-tiny-patch4-window7-224")
SwinModel = SwinModel.from_pretrained('microsoft/swin-tiny-patch4-window7-224').cuda()

Image_Features = []
Image_Features_Test = []

for i in range(len(Task1_Train)):
    if(i%100 == 0):
        print(i)
    image = Image.open(Task1_Train['filename'][i])
    inputs = image_processor(image, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = SwinModel(**inputs)
    last_hidden_states = outputs.pooler_output.detach().cpu().numpy()
    Image_Features.append(last_hidden_states)
    
for i in range(len(Task1_Dev)):
    if(i%100 == 0):
        print(i)
    image = Image.open(Task1_Dev['filename'][i])
    inputs = image_processor(image, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = SwinModel(**inputs)
    last_hidden_states = outputs.pooler_output.detach().cpu().numpy()
    Image_Features_Test.append(last_hidden_states)

preprocessor_config.json:   0%|          | 0.00/255 [00:00<?, ?B/s]

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


config.json:   0%|          | 0.00/71.8k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/113M [00:00<?, ?B/s]

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
0
100
200
300
400
CPU times: user 3min 53s, sys: 4.81 s, total: 3min 58s
Wall time: 3min 27s


In [7]:
%%time
# Tokenize the text using RoBERTa tokenizer
tokenizer = AutoTokenizer.from_pretrained('GroNLP/HateBERT')
bertmodel = AutoModel.from_pretrained('GroNLP/HateBERT').to(device)

text_embeddings = []
text_embeddings_test = []

for text in Task1_Train['text']:
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length= 256)
    inputs = {key: value.to(device) for key, value in inputs.items()}
    with torch.no_grad():
        hidden_train = bertmodel(**inputs)
    cls_train = hidden_train.pooler_output.detach().cpu().numpy()
    text_embeddings.append(cls_train)

for text in Task1_Dev['text']:
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length= 256)
    inputs = {key: value.to(device) for key, value in inputs.items()}
    with torch.no_grad():
        hidden_train = bertmodel(**inputs)
    cls_train = hidden_train.pooler_output.detach().cpu().numpy()
    text_embeddings_test.append(cls_train)

tokenizer_config.json:   0%|          | 0.00/151 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

CPU times: user 40.6 s, sys: 832 ms, total: 41.5 s
Wall time: 44.1 s


In [8]:
Image_features_numpy = np.concatenate(Image_Features, axis=0)
text_embeddings_numpy = np.concatenate(text_embeddings, axis =0)
Image_features_numpy_test = np.concatenate(Image_Features_Test, axis=0)
text_embeddings_numpy_test = np.concatenate(text_embeddings_test, axis =0)

In [9]:
OverallFeature = np.concatenate((text_embeddings_numpy, Image_features_numpy), axis=1)
OverallFeatureTest = np.concatenate((text_embeddings_numpy_test, Image_features_numpy_test), axis=1)

In [10]:
OverallFeature.shape

(3600, 1536)

In [36]:
X_train, X_temp, y_train, y_temp = train_test_split(Image_features_numpy, Task1_Train['label'].to_list(), test_size=0.2, random_state=42)
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [37]:
from xgboost import XGBClassifier

# Train XGBoost model
model = XGBClassifier(objective='binary:logistic', eval_metric='logloss')
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, random_state=None, ...)

In [38]:
from sklearn.metrics import accuracy_score

# Make predictions on the validation set
y_valid_pred = model.predict(X_valid)

# Evaluate model on the validation set
accuracy_valid = accuracy_score(y_valid, y_valid_pred)
print(f'Validation Accuracy: {accuracy_valid}')

# Make predictions on the test set
y_test_pred = model.predict(X_test)

# Evaluate model on the test set
accuracy_test = accuracy_score(y_test, y_test_pred)
print(f'Test Accuracy: {accuracy_test}')

Validation Accuracy: 0.7166666666666667
Test Accuracy: 0.7583333333333333


In [105]:
Results = model.predict(OverallFeatureTest)

In [91]:
import json 

with open('submission.json', 'w') as json_file:
    for i in range(len(Task1_Dev)):
        number = str(Task1_Dev['index'][i])
        predictionsDictionary = {'index': int(number), 'prediction': int(y_test_pred[i]) + 1}
        json.dump(predictionsDictionary, json_file)    
        json_file.write('\n')      

In [66]:
inputs

<tf.Tensor: shape=(2880, 768), dtype=float32, numpy=
array([[-0.21324646, -0.17203829, -0.15404457, ...,  0.01145481,
        -0.14891036,  0.19178045],
       [-0.22433202, -0.18623966, -0.16199364, ..., -0.00120689,
        -0.16137813,  0.18154985],
       [-0.23077035, -0.16107172, -0.14161687, ...,  0.00648975,
        -0.1497752 ,  0.19084701],
       ...,
       [-0.21943179, -0.19655891, -0.16512166, ...,  0.02393809,
        -0.14448357,  0.19682409],
       [-0.22068678, -0.16488023, -0.14218757, ...,  0.01689866,
        -0.14792164,  0.18188976],
       [-0.23724668, -0.18789695, -0.15837544, ...,  0.0067805 ,
        -0.1508908 ,  0.19757459]], dtype=float32)>

In [68]:
X_train.shape

(2880, 768)

In [39]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Convert numpy arrays to TensorFlow tensors
inputs = tf.convert_to_tensor(X_train, dtype=tf.float32)
targets = tf.convert_to_tensor(y_train, dtype=tf.int32)
val_inputs = tf.convert_to_tensor(X_valid, dtype=tf.float32)
val_targets = tf.convert_to_tensor(y_valid, dtype=tf.int32)

# Define the MLP model
model = models.Sequential([
    layers.Dense(512, activation='relu', input_shape=(768,)),
    layers.Dropout(0.2),  # Dropout for regularization
    
    layers.Dense(2, activation='softmax')  # Output layer with 2 units for binary classification
])

# Compile the model
model.compile(optimizer= tf.keras.optimizers.Adam(learning_rate=0.0001),  # Stochastic Gradient Descent
              loss= 'sparse_categorical_crossentropy',  # Use cross-entropy loss for binary classification
              metrics=['accuracy'])

# Training the model
model.fit(inputs, targets, epochs=50, batch_size=32, shuffle=True, validation_data=(val_inputs, val_targets))

# Evaluate the model (Optional)
test_loss, test_accuracy = model.evaluate(inputs, targets)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

Epoch 1/50
90/90 [==============================] - 1s 5ms/step - loss: 0.6573 - accuracy: 0.6257 - val_loss: 0.5755 - val_accuracy: 0.6806
Epoch 2/50
90/90 [==============================] - 0s 3ms/step - loss: 0.5599 - accuracy: 0.7139 - val_loss: 0.5545 - val_accuracy: 0.7083
Epoch 3/50
90/90 [==============================] - 0s 3ms/step - loss: 0.5136 - accuracy: 0.7556 - val_loss: 0.5594 - val_accuracy: 0.6917
Epoch 4/50
90/90 [==============================] - 0s 3ms/step - loss: 0.4738 - accuracy: 0.7740 - val_loss: 0.5558 - val_accuracy: 0.7194
Epoch 5/50
90/90 [==============================] - 0s 3ms/step - loss: 0.4503 - accuracy: 0.7976 - val_loss: 0.5680 - val_accuracy: 0.6917
Epoch 6/50
90/90 [==============================] - 0s 3ms/step - loss: 0.4182 - accuracy: 0.8135 - val_loss: 0.5603 - val_accuracy: 0.7028
Epoch 7/50
90/90 [==============================] - 0s 3ms/step - loss: 0.3929 - accuracy: 0.8344 - val_loss: 0.5695 - val_accuracy: 0.7028
Epoch 8/50
90/90 [==

In [ ]:
import lightgbm as lgb

# lgb hyper-parameters
params = {'metric': 'logloss',
          'num_leaves': 256,
          'learning_rate': 0.005,
          'feature_fraction': 0.75,
          'bagging_fraction': 0.75,
          'bagging_freq': 5,
          'force_col_wise': True,
          'random_state': 44}

# lgb train and valid dataset
dtrain = lgb.Dataset(X_train, y_train)
dvalid = lgb.Dataset(X_valid, y_valid)

# Train LightGBM model
lgb_model = lgb.train(params=params,
                      train_set=dtrain,
                      num_boost_round=2400,
                      valid_sets=[dtrain, dvalid],  # Use valid_sets instead of validation_set
                      early_stopping_rounds=200,
                      verbose_eval=100)

In [17]:
Results = model.predict(OverallFeatureTest)

14/14 [==============================] - 0s 1ms/step


In [21]:
import json 

with open('submission.json', 'w') as json_file:
    for i in range(len(Task1_Dev)):
        predictionsDictionary = {'index': int(Task1_Dev['filename'][i].split('/')[-1][:5]), 'prediction': int(Results[i].argmax())}
        json.dump(predictionsDictionary, json_file)    
        json_file.write('\n')     

In [54]:
inputs2 = tf.convert_to_tensor(X_valid, dtype=tf.float32)
targets3 = tf.convert_to_tensor(y_valid, dtype=tf.int32)

# Evaluate the model (Optional)
test_loss, test_accuracy = model.evaluate(inputs2, targets3)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

12/12 [==============================] - 0s 2ms/step - loss: 1.1388 - accuracy: 0.7194
Test Accuracy: 71.94%
